In [83]:
from selenium.webdriver import Chrome
import pandas as pd
import html5lib
import time
import numpy as np
from math import ceil
import re

In [84]:
webdriver = "chromedriver_win32/chromedriver.exe"
driver = Chrome(webdriver)

# Open a final bets page and copy paste the link below
my_url = "https://rbws.mybookie.ag/PlayerInteraction/AccountManagement2016-07/Account.aspx?SSC=8001&stoken=2a42977cd62dd340&lng=en-US&go=1%2c1%2c1"
driver.get(my_url)

In [90]:
risk_raw = driver.find_elements_by_xpath("//div[@class='tableTotalRBReport']//div[@class='cellRiskRBReport']")
ret_raw = driver.find_elements_by_xpath("//div[@class='tableTotalRBReport']//div[@class='cellReturnRBReport']")
net_raw = driver.find_elements_by_xpath("//div[@class='tableTotalRBReport']//div[@class='cellNetRBReport']")
date_raw = driver.find_elements_by_xpath("//div[contains(@class,'sDvTableCell sDvShowTableMobile')]//span[not(@*)]")
odds = []
risk= []
date = []
win = []
for x in range(len(date_raw)):
    risk.append(risk_raw[x].text)
    date.append(date_raw[x].text)
    # Since there are no odds or results, calculate it using return
    if float(ret_raw[x].text) == 0:
        odds.append(0)
        if float(net_raw[x].text) < 0:
            win.append('N')
        else:
            win.append('R')
    else:
        odds.append(float(ret_raw[x].text)/float(risk[x]))
        win.append('Y')

In [91]:
full_event = driver.find_elements_by_xpath("//span[@class='DetailData']")
event = []
selection = []
for x in range(len(full_event)):
    event.append(re.search(r"^(.*)(?= USD)",full_event[x].text).group(1))
    selection.append(re.search(r"\((.*)\)",full_event[x].text).group(1))

In [93]:
df = pd.DataFrame()
df['Date'] = date
df['Selection'] = selection
df['Event'] = event
df['Bookmaker'] = 'MyBookie'
df['Bet Type'] = ''
df['Tipper'] = ''
df['Live Bet'] = ''
df['Score'] = ''
df['Sport'] = 'Horse racing'
df['My Var'] = ''
df['Stake'] = risk
df['Odds'] = odds
df['Win'] = win
df['Bonus Bet'] = 'N'

cols = ['Date', 'Bookmaker', 'Sport', 'Selection', 'Bet Type', 'Tipper', 'My Var','Event', 'Live Bet',
       'Score', 'Stake', 'Odds', 'Bonus Bet', 'Win']

df = df[cols]

In [94]:
df.head()

,Date,Bookmaker,Sport,Selection,Bet Type,Tipper,My Var,Event,Live Bet,Score,Stake,Odds,Bonus Bet,Win
0,2020/01/21,MyBookie,Horse racing,3: 5/X/X,,,,Exeter Race #6,,,5.00,0.0,N,N
1,2020/01/21,MyBookie,Horse racing,7: 5/X/X,,,,Exeter Race #5,,,5.00,0.0,N,N
2,2020/01/21,MyBookie,Horse racing,8: 5/X/X,,,,Exeter Race #4,,,5.00,0.0,N,N
3,2020/01/21,MyBookie,Horse racing,8: 5/X/X,,,,Exeter Race #3,,,5.00,1.7,N,Y
4,2020/01/21,MyBookie,Horse racing,2: 5/X/X,,,,Newcastle Race #3,,,5.00,3.7,N,Y


In [95]:
df.to_csv('horse_odds.csv', index=False,header=False)